---
title: "Quality Control: Relatedness & Population Stratification"
format:
  html:
    code-fold: false
    toc: true
  ipynb:
    toc: true
    number-sections: false
bibliography: references/references_4.bib
---

::: {.callout-note title="Important notes for this notebook" icon=false}

As we have already mentioned, quality control (QC) analyses are crucial to ensure the reliability and validity of the results. In this section, we will explore confounding factors in GWAS that can lead to spurious associations if not properly controlled, including: 

-  Relatedness  
- Population Stratification/Structure

## Learning outcomes

- **Discuss** confounding factors that lead to spurious associations
- **Detect** related individuals
- **Identify** population substructure
- **Correct** for population stratification 

## How to make this notebook work

In this notebook, we will both use `R` and `bash command line` programming languages. Remember to **change the kernel** whenever you transition from one language to the other (`Kernel --> Change Kernel`) indicated by the languages' images.
* We will first run `Bash` commands.  
  
<img src="Images/bash.png" alt="Bash" width="80"> Choose the Bash kernel

:::

## A. Relatedness

Relatedness refers to the presence of genetically related individuals in a study sample, which can introduce confounding factors into association analyses. Since most statistical methods assume that samples are independent, including related individuals, like siblings, violates this assumption. Addressing relatedness is crucial as a quality control step to ensure that genomes in the dataset are no more similar than expected for unrelated individuals. This process is essential to preserve the validity of the study's findings.

While all humans share some genetic relatedness, the degree of relatedness varies among individuals. In genome-wide association studies (GWAS), these variations must be accounted for to avoid biasing the statistical associations between genotypes and phenotypes. **Cryptic relatedness** (genetic relatedness of individuals not expected to be in a family relationship) can interfere with the association analysis [@voight2005confounding]. 

In a **family-based study** (e.g., parent-offspring), you do not need to remove related pairs, but the statistical analysis should account for family relatedness. However, for a **population-based study**, we suggest using the KING estimator [@KING2010] with a threshold of 0.06.

Differently from the IBD estimator (called pi_hat $\hat{\pi}$), which assumes that individuals come from homogeneous population, KING derives a kinship estimate for a pair of individuals without reference to the population allele frequencies. Instead, it gets the relatedness information from the difference between the counts of loci where both individuals are heterozygotes and counts of loci where they are different homozygotes, normalized by the sum of the heterozygous loci of the individuals. This makes KING robust to population structure and useful to check genotyping quality.

Thus, values for KING are

- 0.25 for parent-child or full siblings: they share exactly 50% of their alleles by descent (IBD). For full siblings it can vary slightly (e.g. recombination events).
- 0.125 for half siblings: they have 25% IBD (one parent is common).
- 0.065 for 1st cousins: IBD is 12.5% since they share one set of grandparents and not parents.

KING values need to be multiplied by 2 to match the corresponding IBD probability

<img src="Images/bash.png" alt="Bash" width="40"> 

We use `ln -sf` to link the data folder and create a directory for output files. 

In [ ]:
ln -sf ../Data
mkdir -p Results/GWAS4

### Analysis with PLINK

We will use the set of independent SNPs (generated in the previous section) and the `--make-king-table` option in PLINK2 for our analysis, so that we generate pairwise KING estimators. Note that we use version 2 of PLINK because this one has implemented the KING estimator.

The HapMap dataset is known to contain parent-offspring relationships. We will check the KING estimator values and exclude all individuals above a chosen threshold (to remove at least related individuals).

In [ ]:
plink2 --bfile Results/GWAS3/HapMap_3_r3_7 \
       --extract Results/GWAS3/indepSNP.prune.in --make-king-table \
       --out Results/GWAS4/KING

The table from PLINK2 contains various things: 

- `FID1`: Family ID of the first individual.
- `IID1`: Individual ID of the first individual.
- `FID2`: Family ID of the second individual.
- `IID2`: Individual ID of the second individual.
- `NSNP`: The number of SNPs used to calculate the kinship between the two individuals.
- `HETHET`: The number of heterozygous sites used in the kinship calculation (often useful for checking genotyping quality).
- `IBS0`: The number of identical-by-state (IBS) 0 SNPs, which are variants where the two individuals do not share the same allele.
- `KINSHIP`: The kinship coefficient (or pihat) between the two individuals, which is the proportion of alleles shared by descent.

:::{.callout-important title="Stop - Read - Solve"}

- What assumption in association studies is violated when individuals are related?
- How would you recognize if the same sample is duplicated in your dataset?
- Are there closely related individuals? How many individuals show close relatedness (`KING > 0.2`)?
- What relationships are found by PLINK?

:::

In [ ]:
# Write your code here (add more cells if needed)

:::{.callout-tip title="Solution" collapse="true"}

- If you have related individuals, you will lose the independence of individuals at each SNP. This is a problem because many models used in GWAS analysis do not include handling family relationships.
- Calculating the KING coefficient and looking for a value close to 0.5.
- We will look for unique values in column 8 (`KINSHIP`), rounding to two decimals
- We will examine `KINSHIP` values in column 8. 



 Let's print the first few rows of `KING.kin0` and then, the unique relatioship categories in the file: 

In [ ]:
cat Results/GWAS4/KING.kin0 | head -5

Now, let's calculate how many samples are in a pair with KING `KINSHIP` parameter (column 8) larger than 0.06, 0.12, 0.2 (to account for small variations in relevant thresholds). Note that the file contains one individual of a pair, so some samples might be shown many times. Thus we make those samples unique.

In [ ]:
awk '{ if ($8 > 0.06) print $2}' Results/GWAS4/KING.kin0 | sort | uniq | wc -l

In [ ]:
awk '{ if ($8 > 0.12) print $2}' Results/GWAS4/KING.kin0 | sort | uniq | wc -l

In [ ]:
awk '{ if ($8 > 0.2) print $2}' Results/GWAS4/KING.kin0 | sort | uniq | wc -l

There are 76 individuals with KINSHIP larger than all thresholds. So we might have 76 half siblings, cousins and siblings mixed in the data. There are potentially some family ties in this data!

:::

---------------

<img src="Images/R.png" alt="R" width="40"> Switch to the R-GWAS kernel.


:::{.callout-important title="Stop - Read - Solve"}

Write some R code to calculate the summary of the distribution of `KINSHIP` values. What do you observe?

hint: use the `hist()` command.
:::

In [ ]:
# Write your code here 

:::{.callout-tip title="Solution" collapse="true"}
Many parent-offspring (PO) relationships have a parameter close to 0.5, and the maximum value for unrelated (UN) individuals is also unexpectedly high relatedness.


We read in the matrix (Note that the header starts with # in the file `KING.kin0`, so we need `comment.char = '|'` to avoid the header being excluded as a comment) 

In [ ]:
# Generate a plot to assess the type of relationship.
suppressMessages(suppressWarnings(library(ggplot2)))

options(repr.plot.width = 9, repr.plot.height = 4)

# Read data into R 
relatedness <- read.table("Results/GWAS4/KING.kin0", header=TRUE, comment.char = '|')

head(relatedness)

Now we look at how many pairs have specific relatedness values. You can see there are around 100 pairs of cousins/half siblings/siblings. Those correspond to the 76 unique samples observed before.

In [ ]:
hist.king <- ggplot(relatedness, aes(x=relatedness[,8])) +
  geom_histogram(binwidth = 0.02, col = "black", fill="tomato") + 
  labs(title = "Histogram of relatedness (KING)") + 
  xlab("KING kinship") + 
  ylab("Log Frequency") + 
  theme_bw() +
  scale_y_log10() +
  theme(axis.title=element_text(size=14), 
        axis.text=element_text(size=13),
        plot.title=element_text(size=15)) 

#Extract coordinates of the plot
bin_data <- ggplot_build(hist.king)$data[[1]]

#Adding text labels to  each bar, 
#accounting for log scale
hist.king + 
  geom_text(data = bin_data, 
            aes(x = xmin + (xmax - xmin) / 2, 
                y = log10(count+1),  # Apply log10
                label = count), 
                vjust = -5, #Vertical adjustment of text
                size = 4, 
                color = "black")

:::

---

We can also plot the number of SNPs used to calculate relatedness VS the KING relatedness itself. Pairs with very low number of SNPs and anomalous KING estimator might simply contain badly genotyped sample(s). This is not at all our case.

In [ ]:
# Relatedness plot
plot.relatedness <- ggplot(relatedness) +
  geom_point(aes(x=NSNP, y=KINSHIP), size=5, alpha=.25) + 
  ylim(-.1,.4) +
  labs(x = "Number of SNPs used", y = "KING kinship", title = "Check for genotyping quality") + 
  theme_bw() +
  theme(axis.title=element_text(size=14), 
        axis.text=element_text(size=13), 
        legend.text= element_text(size=13), 
        legend.title=element_text(size=14), 
        plot.title=element_text(size=15))

show(plot.relatedness)

:::{.callout-important title="Stop - Read - Solve"}
Use `PLINK` options to filter out related individuals, using a relatedness threshold of 0.1. 

- First of all you need to create a file with the IDs of the samples to remove. You can use the command `awk` as we did before in this exercise.

- Afterwards you need PLINK with the option `--remove` to use the file with IDs. Name the output `--out Results/GWAS4/king_exercise`.

:::

<img src="Images/bash.png" alt="Bash" width="40"> Switch to the Bash kernel.

In [ ]:
# Write your code here (PLINK command)


:::{.callout-tip title="Solution" collapse="true"}

We use `awk` to print IDs to remove and then the option `--remove` of PLINK. 

<img src="Images/bash.png" alt="Bash" width="40"> We print out all family and sample IDs for which the column KINSHIP (column 8) is >0.1. Note we print both samples in the pair with high relatedness, using the `awk` command twice.

In [ ]:
awk '{ if ($8 > 0.1) print $1, $2 }' Results/GWAS4/KING.kin0 > Results/GWAS4/filter_king.txt

Now each sample is identified uniquely ba family and sample ID. Some samples are repeated more than once because we have selected the first sample in related pairs, so same samples appear more than once. We use the option `--remove` to produce a new set of `plink` files. We will be warned about sample repetitions because we have been having some duplicates from the KINSHIP pairs

In [ ]:
plink --bfile Results/GWAS3/HapMap_3_r3_7 \
    --make-bed \
    --remove Results/GWAS4/filter_king.txt \
    --out Results/GWAS4/king_exercise

We have gone from 161 to 85 samples, removing 76 related ones.

:::

***

<img src="Images/bash.png" alt="Bash" width="40"> Switch to the Bash kernel.

So far, you’ve learned how to filter individuals using KING kinship thresholds. Now, we’ll demonstrate that most of the relatedness was due to parent-offspring relationships. We will achieve this by using the `--filter-founders` option to create a new filtered dataset that includes only the founders (individuals without listed parents in the data). This generates `.bed`, `.bim`, and `.fam` files, which are then used to create a `.kin0` file containing relatedness values. Note again we use `plink2` to create the KING matrix.

In [ ]:
plink --bfile Results/GWAS3/HapMap_3_r3_7 \
    --filter-founders \
    --make-bed \
    --out Results/GWAS4/HapMap_3_r3_8 \
    --silent

plink2 --bfile Results/GWAS4/HapMap_3_r3_8 \
    --extract Results/GWAS3/indepSNP.prune.in \
    --make-king-table \
    --out Results/GWAS4/KING_founders

<img src="Images/R.png" alt="R" width="40"> Switch to the R kernel.

Let's plot the results from the `KING_founders.kin0` file. We can see how there is only 1 pair with high kinship amongst the founders.

In [ ]:
suppressMessages(suppressWarnings(library(ggplot2)))

relatedness <- read.table("Results/GWAS4/KING_founders.kin0", header=TRUE, comment.char = '|')

hist.king <- ggplot(relatedness, aes(x=relatedness[,8])) +
  geom_histogram(binwidth = 0.02, col = "black", fill="tomato") + 
  labs(title = "Histogram of relatedness (KING) in founders-only data") + 
  xlab("KING kinship") + 
  ylab("Log Frequency") + 
  theme_bw() +
  scale_y_log10() +
  theme(axis.title=element_text(size=14), 
        axis.text=element_text(size=13),
        plot.title=element_text(size=15)) 

#Extract coordinates of the plot
bin_data <- ggplot_build(hist.king)$data[[1]]

#Adding text labels to  each bar, 
#accounting for log scale
hist.king + 
  geom_text(data = bin_data, 
            aes(x = xmin + (xmax - xmin) / 2, 
                y = log10(count+1),  # Apply log10
                label = count), 
                vjust = -10, #Vertical adjustment of text
                size = 4, 
                color = "black")

:::{.callout-important title="Stop - Read - Solve"}

- **1. Do you observe anything unexpected in the plot?**
- **2. Can you identify the IIDs of the sample pair?** Hint: filter for KINSHIP values.
- **3. Based on the KINSHIP, what is the most likely relationship between the pair?**
- **4. What would you do about it?** Hint: remove one or both in the pair?

:::

In [ ]:
# Write your answer here

In [ ]:
# R code here

:::{.callout-tip title="Solution" collapse="true"}

- **1.** The file `KING_founders.kin0` shows that, after excluding all non-founders, one individual pair removed in the exercise remains in the HapMap data. 
- **2.** Below, we can look at the individual from the table by setting KINSHIP > 0.2. Even if labeled as unrelated, it has high relatedness. The IIDs of the pair are NA07045 and NA12813. 


In [ ]:
relatedness[relatedness$KINSHIP > 0.2,]

- **3.** This remaining individual is likely to be related with sibling relationship, even though they were not given the same family identity (FID) in the HapMap data.
- **4.** We don't need to remove both individuals from the dataset (IID1=NA07045; IID2=NA12813). It's enough to remove just one, and the individual with the highest missing rate (lowest quality) could be chosen. Let's calculate the missing rates using PLINK.

:::

***

<img src="Images/bash.png" alt="Bash" width="40"> Switch to the Bash kernel.

In [ ]:
plink --bfile Results/GWAS4/HapMap_3_r3_8 \
    --missing \
    --out Results/GWAS4/HapMap_3_r3_8 \
    --silent

Then, find the two individuals from the output file using `grep` and sorting descendingly by missing rate (EZ, column 6)

In [ ]:
# Let's look at the output
cat Results/GWAS4/HapMap_3_r3_8.imiss | grep -E 'NA07045|NA12813' | sort -k6,6nr

`NA07045` is the individual to be removed. Once this is removed, their offspring can still be used in analyses that don't involve trios, such as family-related studies.

:::{.callout-important title="Stop - Read - Solve"}

- **Plot the distribution of missing rates from `HapMap_3_r3_8`. What is the value for `NA07045`?**
- **Remove `NA07045` from the PLINK files**
  
Hint: 

- Use the `.imiss` file to plot a histogram of the missing scores from the file `HapMap_3_r3_8.imiss` in `R` (check previous notebooks for help). Is the score of `NA07045` amongst the higher/lower ones? You can add a vertical line in the histogram representing the value for `NA07045` using `geom_vline`. 
- Create a file with the sample ID to remove and use the `--remove` option in PLINK. 

:::

<img src="Images/R.png" alt="R" width="40"> Switch to the R kernel.

In [ ]:
# Write your code here

:::{.callout-tip title="Solution" collapse="true"}
We can extract the column from the file and observe in the histogram that the individual `NA07045` has a missing rate closer which is in the bulk of other samples.


In [ ]:
suppressMessages(suppressWarnings(library(ggplot2)))

options(repr.plot.width = 9, repr.plot.height = 4)

# Read data into R 
smiss <- read.table("Results/GWAS4/HapMap_3_r3_8.imiss", header=T)

head(smiss)

hist.relatedness <- ggplot(data=smiss, aes(F_MISS)) +
  geom_histogram(col = "black", fill = "tomato") + 
  geom_vline(xintercept = 0.002384, color = "red", linetype = "dashed", linewidth = 1) +
  labs(title = "Histogram of missing rates") + 
  xlab("Missing rate") + 
  ylab("Frequency") + 
  theme_bw() +
  theme(axis.title=element_text(size=14), axis.text=element_text(size=13),plot.title=element_text(size=15))

show(hist.relatedness)


<img src="Images/bash.png" alt="Bash" width="40"> Switch to the Bash kernel.

To remove the individual from the data, we create a file containing its number and ID, and provide it to PLINK with the option `--remove`

In [ ]:
echo "13291  NA07045" > Results/GWAS4/id_remove.txt

In [ ]:
plink --bfile Results/GWAS4/HapMap_3_r3_8 \
    --remove Results/GWAS4/id_remove.txt \
    --make-bed \
    --out Results/GWAS4/HapMap_3_r3_9 \
    --silent

:::

***

## B. Population Stratification

Population stratification presents a significant source of systematic bias in GWAS, arising **when subpopulations exhibit systematic differences in allele frequencies**. Research indicates that even subtle degrees of population stratification can exist within a single ethnic population [@abdellaoui2013population]. Thus, testing and controlling for the presence of population stratification is an essential QC step.

The population structure (or in other words, the ancestral relationship of the populations) is a so-called **confounding factor**. This means that it affects both the dependent and independent variables, as shown in the figure below, where both the genotype and traits are influenced by population structure (e.g., the distribution of north and south European individuals in the PCA space and the height of those individuals).

![](Images/ancestryBias.png){fig-align="center" width=400px}

Why is a bias introduced? Population structure can influence allele frequencies and produce false positives/negatives when doing association testing. Graphically, consider the example in the figure below. Case and control for population 1 and 2 have specific MAFs. Removing the barrier between populations will change MAFs drastically for the two confitions.

![](Images/popStruct.png){fig-align="center" width=400px}

The same problem arises in population studies without Case-control categories. Imagine having a population of randomly sampled individuals, each from a different ethnicity (the blue and red minor alleles in the example below). The final group of individuals will have a different proportion of MAFs depending on the sampling of various ethnicities.

![](Images/popSampling.png){fig-align="center" width=400px}

There are several methods to correct for population stratification [@price2010new, @price_principal_2006]. Here, we illustrate a method integrated into PLINK: the **multidimensional scaling (MDS)** approach. MDS calculates the genome-wide average proportion of shared alleles between any pair of individuals to generate quantitative indices (components) of the genetic variation for each individual. **The individual component scores can be visualized to identify groups of genetically similar individuals.** For instance, in a genetic study including subjects from Asia and Europe, MDS analysis would reveal that Asians are genetically more similar to each other than to Europeans and Africans. The figure below shows another example of MDS using HapMap, Genome diversity project, and authors' data:

![Example of MDS plot of the first two MDS components scores on integrated datasets in @somers_genetic_2017. Here TWN, a past isolated population in the Netherlands, is separated from the other individuals from the same country (NDL). Sardinians, notoriously different from continental Italians, are also grouped far away from them. Central European populations colocalize as expected.](./Images/MDSexample2.png){width=500px label=fig-MDSexample}

To investigate which individuals the generated component scores deviate from in the target population, plotting the scores of the dataset under investigation and a population of known ethnic structure (e.g., HapMap/1KG data) is helpful: this step is called **anchoring** [@rietveld2013gwas]. This enables the researcher to obtain ethnic information on their data and to determine possible ethnic outliers. For example, in the figure above, if TSI (Tuscans from Italy) is the anchor population, one can hypothesize that the yellow dots might be ethnically similar (as in the example).

Outliers identified based on MDS analysis should be excluded from further analyses. Following their removal, a new MDS analysis must be conducted, and its primary components are utilized as covariates in association tests to correct for any residual population stratification within the population. The number of components to include depends on the population structure and sample size (usually 10-20). 

The MDS from @cortellari_climatic_2021 shows a distinct goat population outlier. The second axis is dominated by this outlier, obscuring structure in the other populations. Removing the outlier reveals a clearer structure among the remaining populations.

![](./Images/MDSout.png){fig-align="center" width=700px}


### Analysis

We aim to merge the HapMap and 1000GP datasets, using 1000GP Phase I as the anchor for HapMap. Our goal is to check if we can identify the ethnicity of the HapMap data based on the ethnicities in the 1000GP dataset. There are several steps to ensure compatibility between the datasets, so stay with us!

#### 1000GP data download

Here are some commands to download and convert the 1000GP data for GWAS analysis. **You don’t need to run them**, as we’ve already processed the data.

- 1000 Genomes Project - Phase I: genetic information for 629 individuals from various ethnic groups (>60GB). *Phase III is now available, and we recommend using it for research purposes.*


```{.bash}
wget ftp://ftp-trace.ncbi.nih.gov/1000genomes/ftp/release/20100804/ALL.2of4intersection.20100804.genotypes.vcf.gz
```

The data was converted from the `vcf` file (Variant Call Format) to plink format (`bim`, `fam`, `bed`):

```{.bash}
plink2 --vcf ALL.2of4intersection.20100804.genotypes.vcf.gz --make-bed --out ALL.2of4intersection.20100804.genotypes
```

:::{.callout-note}

The 1000 Genomes data downloaded above is rather large so the commands are not executable and are shown for reference only. To save time, we’ve provided the `.bed`, `.bim`and `.fam` files in the `Data` folder.

:::

Let's unzip the files and see how many samples we have.

In [ ]:
unzip -o Data/1000genomes.zip -d Results/GWAS4
# count lines in fam 
wc -l Results/GWAS4/1000genomes.genotypesA.fam

We have a subset of **37 individuals**. Now, let's explore the `bim` file. 

In [ ]:
cat Results/GWAS4/1000genomes.genotypesA.bim | head -5

One should note that the file `1000genomes.genotypes.bim` contains SNPs without an rs-identifier (or Reference SNP cluster ID). The missing rs-identifiers (noted as `.`) are not a problem for this tutorial. However, for good practice, we will assign unique identifiers to the SNPs (using available information):

In [ ]:
plink --bfile Results/GWAS4/1000genomes.genotypesA \
    --set-missing-var-ids @:#[b37]\$1,\$2 \
    --make-bed \
    --out Results/GWAS4/1000genomes.genotypesA_no_missing_IDs \
    --silent

Now, let’s visualize the data to check the SNP names assigned. These are derived from the format `@:#[b37]\$1,\$2` in the command above, which PLINK interprets as `chromosome:locus[b37]Allele1,Allele2`: 

In [ ]:
# Show changes on the bim file
cat Results/GWAS4/1000genomes.genotypesA_no_missing_IDs.bim | head -5

Pretty neat, right?

#### QC on 1000GP data
As we covered in the GWAS3 notebook, it's important to account for missingness, sex discrepancies, and minor allele frequency. We'll apply standard QC thresholds to the 1000 GP data before merging it with HapMap data. 

In [ ]:
plink --bfile Results/GWAS4/1000genomes.genotypesA_no_missing_IDs \
    --geno 0.02 \
    --make-bed \
    --out Results/GWAS4/1kG_MDS --silent

plink --bfile Results/GWAS4/1kG_MDS \
    --mind 0.02 \
    --make-bed \
    --out Results/GWAS4/1kG_MDS2 --silent
    
plink --bfile Results/GWAS4/1kG_MDS2 \
    --maf 0.05 \
    --make-bed  \
    --out Results/GWAS4/1kG_MDS3 --silent

#### SNPs matching between datasets

N.B: Ensure that the datasets you want to merge share the same genomic build! Otherwise, you'll need to include a liftover step.


We want to only consider SNPs that both datasets have in common.  First, extract SNP names from the HapMap data and filter the 1000GP data to include only matching SNPs.


In [ ]:
#Print out SNPs from the HapMap data
awk '{print$2}' Results/GWAS4/HapMap_3_r3_9.bim > Results/GWAS4/HapMap_SNPs.txt

#Extract the HapMap SNPs from the 1000GP data
plink --bfile Results/GWAS4/1kG_MDS3 \
    --extract Results/GWAS4/HapMap_SNPs.txt \
    --make-bed \
    --out Results/GWAS4/1kG_MDS4

This is how part of the list of SNP names looks like:

In [ ]:
cat Results/GWAS4/HapMap_SNPs.txt | head -5

Now we take the variants from the reduced 1000GP data, and go the other way around. We extract 1000GP variants from the HapMap data. In other words, the two extraction passages will intersect the SNPs. Below is the code to use the SNPs of the 1000GP data to reduce the HapMap data.

In [ ]:
#Print out SNPs from the HapMap data
awk '{print$2}' Results/GWAS4/1kG_MDS4.bim > Results/GWAS4/1kG_MDS4_SNPs.txt

#Extract the HapMap SNPs from the 1000GP data
plink --bfile Results/GWAS4/HapMap_3_r3_9 \
    --extract Results/GWAS4/1kG_MDS4_SNPs.txt \
    --make-bed \
    --out Results/GWAS4/HapMap_MDS

:::{.callout-important title="Stop - Read - Solve"}

**Look at the two outputs below a bit more carefully. The names are all matching. Is there any other problem?**

:::

In [ ]:
head Results/GWAS4/HapMap_MDS.bim

In [ ]:
head Results/GWAS4/1kG_MDS4.bim

#### Build matching

Genomic data is based on a reference genome, and our datasets use different human reference versions. Since the reference genome improves over time, SNP positions may differ between datasets from different versions.

We extract SNP names and positions from the HapMap data and align the 1000GP data to match these SNPs using the `--update-map` option in PLINK.

In [ ]:
#Extract the HapMap variant coordinates
awk '{print$2,$4}' Results/GWAS4/HapMap_MDS.bim > Results/GWAS4/buildhapmap.txt

This is how the list of SNPs look like:

In [ ]:
cat Results/GWAS4/buildhapmap.txt | head -5

We run PLINK to update the 1000GP variant coordinates based on HapMap, ignoring the warning about unsorted positions: 

In [ ]:
plink --bfile Results/GWAS4/1kG_MDS4 \
    --update-map Results/GWAS4/buildhapmap.txt \
    --make-bed \
    --out Results/GWAS4/1kG_MDS5

#### Merging datasets and performing MDS

Before merging the HapMap and 1000 Genomes datasets, we ensure compatibility through 3 steps:

1. Verify the reference genome is compatible in both datasets.
2. Align SNP orientations (strand) across datasets.
3. Remove SNPs that still differ after these steps.

The next steps are technical but ensure the datasets correspond correctly.


**1.** We’ve matched SNP positions, but we also need to ensure the reference alleles align. Remember that most PLINK analyses **consider the A1 allele (typically the minor allele) as the reference allele**, which is logical when dealing exclusively with biallelic variants.

Below, we generate a list of SNPs ID and 'reference alleles' (corresponding to A1, column 5 of the `.bim` file) from 1000GP.

In [ ]:
#Extract variant coordinates and reference alleles from 1000GP data
awk '{print$2,$5}' Results/GWAS4/1kG_MDS5.bim > Results/GWAS4/1kg_ref-list.txt

How the list looks like:

In [ ]:
head -5 Results/GWAS4/1kg_ref-list.txt

Then, we assign them to the HapMap data `--ref-allele` option. Note a lot fo warnings in A1 allele assignment. This is usually because you have a strand issue, where the data uses a complement

In [ ]:
plink --bfile Results/GWAS4/HapMap_MDS  \
    --make-bed \
    --reference-allele Results/GWAS4/1kg_ref-list.txt  \
    --out Results/GWAS4/HapMap-adj 

**2.** To resolve strand issues, we flip SNPs found in both datasets with complementary alleles (i.e. they were reported in opposite strands). We generate SNP lists (ID and alleles) for both datasets, identify unique SNPs, and visualize differences in allele reporting. If a SNP is unique but reports alleles differently, it will appear twice. Below are examples of SNPs with strand issues from the 1000GP and HapMap data:

In [ ]:
#print SNP and A1 A2 alleles on files for the two datasets
awk '{print$2,$5,$6}' Results/GWAS4/1kG_MDS5.bim > Results/GWAS4/1kGMDS5_tmp
awk '{print$2,$5,$6}' Results/GWAS4/HapMap-adj.bim > Results/GWAS4/HapMap-adj_tmp

#sort by SNP name to see SNPs with complementary alleles
sort Results/GWAS4/1kGMDS5_tmp Results/GWAS4/HapMap-adj_tmp | uniq -u > Results/GWAS4/all_differences.txt

In [ ]:
head -6 Results/GWAS4/all_differences.txt

**How many of these differences are there?** We can count 604 lines, so 302 SNPs

In [ ]:
wc -l Results/GWAS4/all_differences.txt

Some of these differences might be might be due to strand issues.

Let's look at this variant `rs10006274`. **Will it be flipped in the HapMap dataset?**

The answer is yes! If we look at the reference allele in `1kg_ref-list.txt`, it shows `C`. This means the SNP is on the forward strand in 1000GP (C/T) and on the reverse strand in HapMap (G/A).

In [ ]:
grep rs10006274 Results/GWAS4/1kg_ref-list.txt

In [ ]:
grep rs10006274 Results/GWAS4/all_differences.txt

:::{.callout-important title="Stop - Read - Solve"}

Look at these other SNPs: `rs9614750` and `rs10088098`. 

- **Which ones will have to be flipped?**
- **Is it always the same dataset that must be flipped?**

:::

In [ ]:
# Write your code here 

:::{.callout-tip title="Solution" collapse="true"}
We will first print out the SNPs from the reference file to know which line corresponds to each dataset (since we know we used the 1000 Genomes Project as the reference). If there are strand issues, the SNP will need to be flipped in the dataset that wasn't used as the reference.


In [ ]:
grep rs9614750 Results/GWAS4/1kg_ref-list.txt 

In [ ]:
grep rs9614750 Results/GWAS4/all_differences.txt

For `rs9614750`, the genotype is reported as `A/G` in the 1000GP data, while in HapMap, it is `C/G`. This discrepancy between the two datasets means that the SNP will need to be removed later.


In [ ]:
grep rs10088098 Results/GWAS4/1kg_ref-list.txt 

In [ ]:
grep rs10088098 Results/GWAS4/all_differences.txt

For `rs10088098`, the genotype is reported as `C/T` in the 1000GP data, while in HapMap, it is `G/A`. This means that PLINK will flip the allele, as they are complementary.

:::

---

Now we take only the SNP names and give them to PLINK (option `--flip`), together with the reference genome (option `--reference-allele`):

In [ ]:
## Flip SNPs for resolving strand issues.
# Print SNP-identifier and remove duplicates.
awk '{print$1}' Results/GWAS4/all_differences.txt | sort -u > Results/GWAS4/flip_list.txt

In [ ]:
wc -l Results/GWAS4/flip_list.txt

These are the SNP ID of non-corresponding SNPs (N=302) between the two files. 

In [ ]:
head -5 Results/GWAS4/flip_list.txt

Apply the flipping option: 

In [ ]:
plink --bfile Results/GWAS4/HapMap-adj \
    --flip Results/GWAS4/flip_list.txt \
    --reference-allele Results/GWAS4/1kg_ref-list.txt \
    --make-bed \
    --out Results/GWAS4/corrected_hapmap \
    --silent

:::{.callout-important title="Stop - Read - Solve"}
There might still be problematic SNPs after flipping. 

- **Check if the expected allele flip occurred.**

Hint: use `grep` to find the `rs10006274` and `rs9614750` variants; then compare the alleles assignments in the HapMap file before and after flipping.

:::

In [ ]:
# Write your code here 

:::{.callout-tip title="Solution" collapse="true"}
PLINK attempts to flip all SNPs in the list, but an error occurs when A1 does not match the one in the reference.


Notice how this SNP has changed as we predicted: 

In [ ]:
grep rs10006274 Results/GWAS4/corrected_hapmap.bim

In [ ]:
grep rs10006274 Results/GWAS4/HapMap-adj.bim

`rs10006274` was flipped and has the same strand orientation in both datasets (same alleles in A1 and A2). 

What happened to this one? PLINK attempts to resolve the mismatch by flipping the alleles but throws an error because the complementary alleles do not match the reference!

In [ ]:
grep rs9614750 Results/GWAS4/corrected_hapmap.bim

In [ ]:
grep rs9614750 Results/GWAS4/HapMap-adj.bim

:::

---

:::{.callout-note}
You don't need to flip the 1000GP data because the reference allele (A1) in the 1000GP data already matches the strand orientation used in the HapMap data.

:::

**3.** After flipping SNPs, some differ in their alleles when comparing datasets to each other (e.g. SNP `rs9614750`) and such SNPs must be removed. 

We extract the SNPs from the corrected HapMap data and search for unique SNP (ID, A1, and A2), comparing them with those from the 1000GP data.

In [ ]:
awk '{print$2,$5,$6}' Results/GWAS4/corrected_hapmap.bim > Results/GWAS4/corrected_hapmap_tmp

sort Results/GWAS4/1kGMDS5_tmp Results/GWAS4/corrected_hapmap_tmp | uniq -u  > Results/GWAS4/uncorresponding_SNPs.txt

**How many SNP mismatches are there?**

In [ ]:
wc -l Results/GWAS4/uncorresponding_SNPs.txt

This corresponds to 12 unique SNP IDs with mismatched information.

In [ ]:
head Results/GWAS4/uncorresponding_SNPs.txt 

We extract again the SNP IDs from the file above, and exclude them using the PLINK option `--exclude` in both datasets

In [ ]:
awk '{print$1}' Results/GWAS4/uncorresponding_SNPs.txt | sort -u > Results/GWAS4/SNPs_for_exclusion.txt

plink --bfile Results/GWAS4/corrected_hapmap \
    --exclude Results/GWAS4/SNPs_for_exclusion.txt \
    --make-bed \
    --out Results/GWAS4/HapMap_MDS3 \
    --silent
plink --bfile Results/GWAS4/1kG_MDS5 \
    --exclude Results/GWAS4/SNPs_for_exclusion.txt \
    --make-bed \
    --out Results/GWAS4/1kG_MDS6 \
    --silent

**5.** We can finally merge the data! We provide our dataset (`-bfile`) and the one to add (`--bmerge` option):

In [ ]:
plink --bfile Results/GWAS4/corrected_hapmap \
    --bmerge Results/GWAS4/1kG_MDS6.bed Results/GWAS4/1kG_MDS6.bim Results/GWAS4/1kG_MDS6.fam \
    --allow-no-sex \
    --make-bed \
    --out Results/GWAS4/MDS_merge \
    --silent

### Perform MDS on HapMap-CEU data anchored by 1000 Genomes data.

MDS is typically performed on independent SNPs (pruned SNPs). We have previously identified such SNPs in this course, so we will extract only these SNPs for the analysis.

In [ ]:
plink --bfile Results/GWAS4/MDS_merge \
    --extract Results/GWAS3/indepSNP.prune.in \
    --genome \
    --out Results/GWAS4/MDS_merge \
    --silent

Now, we can use PLINK to run MDS with the option `---mds-plot` specifying the number of components to calculate.

In [ ]:
# mds-plot
plink --bfile Results/GWAS4/MDS_merge \
    --read-genome Results/GWAS4/MDS_merge.genome \
    --cluster \
    --mds-plot 10 \
    --out Results/GWAS4/MDS_merge \
    --silent

For visualization purposes, we downloaded the 1000 Genomes Project (1000GP) panel, which includes individual names and their corresponding population information.

In [ ]:
wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20100804/20100804.ALL.panel -P Results/GWAS4 -q

To determine the population origins of HapMap individuals, we created a new file that combines the 1000GP panel information with the HapMap data, labeling the population HapMap entries as 'OWN'.

In [ ]:
awk '{print$1,$2,"OWN"}' Results/GWAS4/HapMap_MDS.fam > Results/GWAS4/popfile_own.txt
awk '{print$1,$1,$2}' Results/GWAS4/20100804.ALL.panel > Results/GWAS4/20100804.ALL.panel.txt

cat <(echo "FID IID POP") Results/GWAS4/20100804.ALL.panel.txt Results/GWAS4/popfile_own.txt > Results/GWAS4/popfile.txt

:::{.callout-note}

The 1000 Genomes Project (1000GP) categorizes individuals into major continental groups—such as Europeans (EUR), Africans (AFR), Americans (AMR), East Asians (EAS), and South Asians (SAS)—each comprising various subpopulations. We will use this population structure information to visualize and determine the clusters our samples belong to.

![](./Images/1000GP.png){width=800px}

:::

<img src="Images/R.png" alt="R" width="40"> Switch to the R kernel.

Let's visualize population stratification using the multidimensional scaling (MDS) results. 


In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)

suppressMessages(suppressWarnings(library(ggplot2)))

# Read data into R 
data <- read.table(file="Results/GWAS4/MDS_merge.mds",header=TRUE)
pop <- read.table(file="Results/GWAS4/popfile.txt",header=TRUE)
datafile <- merge(data,pop,by=c("FID","IID"))

# Metapopulation information for the population in the 1000GP dataset
superpop <- c(
  "JPT" = "ASN",
  "ASW" = "AFR",
  "CEU" = "EUR",
  "CHB" = "ASN",
  "CHD" = "ASN",
  "YRI" = "AFR",
  "LWK" = "AFR",
  "TSI" = "EUR",
  "MXL" = "AMR",
  "GBR" = "EUR",
  "FIN" = "EUR",
  "CHS" = "ASN",
  "PUR" = "AMR",
  "OWN" = "UN"
)

# add metapopulation info to the table 
datafile$SUPERPOP <- superpop[datafile$POP]

# Plotting                
scatter.mds <- ggplot(datafile, aes(x=C1, y=C2, color=SUPERPOP)) +
  geom_point(size=5, alpha=.4) +
  scale_color_manual(values=c("AFR" = "red", "AMR" = "springgreen4", "ASN" = "gold", "EUR" = "blue", "UN" = "grey" )) +
  xlab("MD Component 1") + 
  ylab("MD Component 2") +
  labs(color="Superpop") +
  theme_bw() +
  theme(axis.title = element_text(size = 14), legend.text = element_text(size = 15), 
        axis.text = element_text(size = 14), legend.title=element_text(size=15))
  
show(scatter.mds)

The HapMap data clusters closely with European populations such as CEU, TSI, IBS, GBR, and FIN, confirming its European composition. Additionally, the absence of distant points indicates no outliers in the HapMap dataset (grey datapoints cluster together).


#### Exclude ethnic outliers

Let's run the scripts to filter population stratification outliers for educational purposes (e.g. imagine a HapMap individual clusters in the lower-right corner with African populations). 

<img src="Images/bash.png" alt="Bash" width="40"> Switch to the Bash kernel.

To identify and exclude ethnic outliers in the HapMap dataset, select individuals falling within specific cut-off thresholds. These thresholds should be determined based on the visualization of the first two dimensions from the multidimensional scaling (MDS) analysis. 

**What values would you select or apply in this context, and why?** Hint: look at the plot above. 

Preview of the MDS results: 

In [ ]:
head -3 Results/GWAS4/MDS_merge.mds | cut -f1-6 -d$'\t'

Based on the position of the target population's cluster in the MDS plot, we will exclude individuals with MDS component 1 values less than 0 and MDS component 2 values less than -0.05.


The selection has to be done for the individuals we want to keep, so the area to be chosen has to be mirrored into >0 and >-0.05. We provide the MDS component values in columns 4 and 5 of the file (corresponding to the first 2 components), and extract the individuals using `--keep` 

In [ ]:
awk '{ if ($4 >0 || $5 >-0.05) print $1,$2 }' Results/GWAS4/MDS_merge.mds > Results/GWAS4/EUR_MDS_merge

plink --bfile Results/GWAS4/HapMap_3_r3_9 \
    --keep Results/GWAS4/EUR_MDS_merge \
    --make-bed  \
    --out Results/GWAS4/HapMap_3_r3_10 \
    --silent

#### Creating covariates for GWAS analysis

The 10 MDS dimensions will be used as covariates in the association analysis in the next tutorial to correct for population stratification. The covariate file is created by removing column 3 (`SOL`, optional metadata) from the MDS output file.

:::{.callout-important title="Stop - Read - Solve"}
**Why are we computing the covariates again?**
:::

In [ ]:
plink --bfile Results/GWAS4/HapMap_3_r3_10 \
    --extract Results/GWAS3/indepSNP.prune.in \
    --genome  \
    --out Results/GWAS4/HapMap_3_r3_10 \
    --silent
plink --bfile Results/GWAS4/HapMap_3_r3_10 \
    --read-genome Results/GWAS4/HapMap_3_r3_10.genome \
    --cluster \
    --mds-plot 10 \
    --out Results/GWAS4/HapMap_3_r3_10_mds \
    --silent

# Change the format of the .mds file into a plink covariate file.
awk '{print $1, $2, $4, $5, $6, $7, $8, $9, $10, $11, $12, $13}' Results/GWAS4/HapMap_3_r3_10_mds.mds > Results/GWAS4/covar_mds.txt

The `covar_mds.txt` file contains the covariates to adjust for residual population stratification.

In [ ]:
head -5 Results/GWAS4/covar_mds.txt

:::{.callout-note title="Wrapping up"}

You have now successfully checked your data for relatedness population stratification. You filtered out the individuals with high relatedness and produced a summary of the population structure using the MDS projection. You will use the MDS coordinates as a proxy for the population structure you want your association testing to be corrected for.  

In the next notebook on **Association Testing**, you will need the following files from the folder `Results/GWAS4/`:
- `HapMap_3_r3_10` (the bfile, i.e., `HapMap_3_r3_10.bed`, `HapMap_3_r3_10.bim`, and `HapMap_3_r3_10.fam`)
- `covar_mds.txt`
which are the HapMap data and the MDS covariates highlighting the population stratification. Those are already available once you have been running this notebook.

Below is a cheat sheet of our new methods of QC. Again, it is important to remember that each method of QC should be justified, which will depend on the nature of the feature you are trying to analyze.


| Step| Command| Function| Thresholds and explanation|
|------------------------------|------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 6: Relatedness | --make-king-table (PLINK2)| Calculates KING kinship of all sample pairs. The KING kinship is more reliable in presence of population structure.| Use independent SNPs (pruning) for this analysis and limit it to autosomal chromosomes only.|
| - | awk + --remove | Sets threshold and creates a list of individuals with relatedness above the chosen threshold (`awk` command). This means that subjects who are related at, for example, KINSHIP > 0.1 (i.e., half siblings, or siblings, or parent-child) can be detected. | Cryptic relatedness can interfere with the association analysis. If you  have a family‐based sample (e.g., parent‐offspring), you do not need to remove related pairs but the statistical analysis should take family relatedness into account. However, for a population-based sample, we suggest using a KINSHIP threshold of 0.06, which is in line with the literature using IBD kinship thresholds (Anderson et al.,  2010; Guo et al.,  2014). |
| 7: Population Stratification | --cluster --mds-plot k | Produces a  k‐dimensional representation of any substructure in the data, based on IBS.| K is the number of dimensions, which needs to be defined  (typically 10). This is an important step of the QC that consists of  multiple proceedings but for reasons of completeness, we briefly refer to  this step in the table. This step will be described in more detail in the section “Controlling for population stratification.” |

:::

# Bibliography